# 📘 Entrenamiento de Modelos Baseline para Predicción Espaciotemporal de Precipitación Mensual STHyMOUNTAIN

Este notebook implementa modelos baseline para la predicción de precipitaciones usando datos espaciotemporales.

In [15]:
# Configuración del entorno (compatible con Colab y local)
import os
import sys
from pathlib import Path

# Detectar si estamos en Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Si estamos en Colab, clonar el repositorio
    !git clone https://github.com/username/ml_precipitation_prediction.git
    %cd ml_precipitation_prediction
    # Instalar dependencias necesarias
    !pip install -r requirements.txt
    !pip install xarray netCDF4 optuna matplotlib seaborn lightgbm xgboost scikit-learn
    BASE_PATH = Path('.')
else:
    # Si estamos en local, usar la ruta actual
    if '/models' in os.getcwd():
        BASE_PATH = Path('..')
    else:
        BASE_PATH = Path('.')

print(f"Entorno configurado. Usando ruta base: {BASE_PATH}")

# Crear directorio para la salida de modelos si no existe
model_output_dir = BASE_PATH / 'models' / 'output'
model_output_dir.mkdir(parents=True, exist_ok=True)
print(f"Directorio para salida de modelos creado: {model_output_dir}")

Entorno configurado. Usando ruta base: ..
Directorio para salida de modelos creado: ../models/output


In [16]:
# 1. Importaciones necesarias
import numpy as np
import pandas as pd
import xarray as xr
import optuna
import pickle
import datetime
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
# 2. Cargar el dataset NetCDF
def load_dataset(file_path):
    """Carga un archivo NetCDF y lo convierte a pandas DataFrame"""
    try:
        # Cargar el archivo NetCDF con xarray
        print(f"Intentando cargar el archivo: {file_path}")
        ds = xr.open_dataset(file_path)
        print("Archivo cargado exitosamente con xarray")
        
        # Mostrar información del dataset cargado
        print("\nInformación del dataset:")
        print(ds.info())
        print("\nVariables disponibles:")
        for var_name in ds.data_vars:
            print(f"- {var_name}: {ds[var_name].shape}")
        
        # Convertir a DataFrame
        df = ds.to_dataframe().reset_index()
        return df, ds
    except Exception as e:
        print(f"Error al cargar el archivo NetCDF: {e}")
        return None, None

# Ruta al dataset
data_file = BASE_PATH / 'data' / 'output' / 'complete_dataset_with_features.nc'
print(f"Buscando archivo en: {data_file}")

# Cargar el dataset
df, ds_original = load_dataset(data_file)

# Verificar si se cargó correctamente
if df is not None:
    print(f"Dataset cargado con éxito. Dimensiones: {df.shape}")
    print("\nPrimeras filas del DataFrame:")
    display(df.head())
else:
    print("No se pudo cargar el dataset. Verificar la ruta y el formato del archivo.")

Buscando archivo en: ../data/output/complete_dataset_with_features.nc
Intentando cargar el archivo: ../data/output/complete_dataset_with_features.nc
Archivo cargado exitosamente con xarray

Información del dataset:
xarray.Dataset {
dimensions:
	time = 530 ;
	latitude = 62 ;
	longitude = 66 ;

variables:
	datetime64[ns] time(time) ;
	float32 latitude(latitude) ;
	float32 longitude(longitude) ;
	float32 total_precipitation(time, latitude, longitude) ;
	float32 max_daily_precipitation(time, latitude, longitude) ;
	float32 min_daily_precipitation(time, latitude, longitude) ;
	float32 daily_precipitation_std(time, latitude, longitude) ;
	float32 month_sin(time, latitude, longitude) ;
	float32 month_cos(time, latitude, longitude) ;
	float32 doy_sin(time, latitude, longitude) ;
	float32 doy_cos(time, latitude, longitude) ;
	float64 elevation(latitude, longitude) ;
	float32 slope(latitude, longitude) ;
	float32 aspect(latitude, longitude) ;

// global attributes:
	:description = ST-HyMOUNTAIN-

,time,latitude,longitude,total_precipitation,max_daily_precipitation,min_daily_precipitation,daily_precipitation_std,month_sin,month_cos,doy_sin,doy_cos,elevation,slope,aspect
0,1981-01-01,4.324997,-74.975006,47.381050,24.706928,0.0,5.825776,0.5,0.866025,0.017202,0.999852,493.784552,89.539551,102.044502
1,1981-01-01,4.324997,-74.925003,40.750824,21.819195,0.0,5.019045,0.5,0.866025,0.017202,0.999852,519.750107,89.867020,73.481674
2,1981-01-01,4.324997,-74.875008,46.338623,26.092327,0.0,5.740223,0.5,0.866025,0.017202,0.999852,248.776045,89.722221,65.916817
3,1981-01-01,4.324997,-74.825005,48.779938,29.421450,0.0,5.611738,0.5,0.866025,0.017202,0.999852,351.415728,86.986130,140.916000
4,1981-01-01,4.324997,-74.775002,38.932945,18.483061,0.0,3.733574,0.5,0.866025,0.017202,0.999852,278.261922,88.273293,18.439939


In [18]:
# 3. Preparación de los datos
if df is not None:
    # Identificar la columna objetivo (precipitación)
    target_column = 'total_precipitation'  # Ajustar si tiene otro nombre en tu dataset
    
    # Ver si existe 'precip_target' o usar 'total_precipitation'
    if 'total_precipitation' in df.columns:
        target_column = 'total_precipitation'
    
    print(f"Columna objetivo identificada: {target_column}")
    
    # Separar variables predictoras y variable objetivo
    feature_cols = [col for col in df.columns if col != target_column and not pd.isna(df[col]).all()]
    
    # Eliminar columnas no numéricas para los modelos (como fechas o coordenadas si no se usan como features)
    non_feature_cols = ['time', 'spatial_ref']
    feature_cols = [col for col in feature_cols if col not in non_feature_cols]
    
    # Eliminar filas con valores NaN
    print(f"Filas antes de eliminar NaN: {df.shape[0]}")
    df_clean = df.dropna(subset=[target_column] + feature_cols)
    print(f"Filas después de eliminar NaN: {df_clean.shape[0]}")
    
    # Separar features y target
    X = df_clean[feature_cols]
    y = df_clean[target_column]
    
    print(f"\nFeatures seleccionadas ({len(feature_cols)}):\n{feature_cols}")
    print(f"\nVariable objetivo: {target_column}")

Columna objetivo identificada: total_precipitation
Filas antes de eliminar NaN: 2168760
Filas después de eliminar NaN: 2168760

Features seleccionadas (12):
['latitude', 'longitude', 'max_daily_precipitation', 'min_daily_precipitation', 'daily_precipitation_std', 'month_sin', 'month_cos', 'doy_sin', 'doy_cos', 'elevation', 'slope', 'aspect']

Variable objetivo: total_precipitation


In [19]:
# 4. División del conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Dimensiones del conjunto de entrenamiento: {X_train.shape}")
print(f"Dimensiones del conjunto de prueba: {X_test.shape}")

# 5. Estandarización de variables predictoras
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Guardar el scaler para uso futuro
with open(model_output_dir / 'scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("Escalador guardado en models/output/scaler.pkl")

Dimensiones del conjunto de entrenamiento: (1735008, 12)
Dimensiones del conjunto de prueba: (433752, 12)
Escalador guardado en models/output/scaler.pkl


In [20]:
# 6. Funciones de evaluación y entrenamiento
def evaluar_modelo(y_true, y_pred):
    """Evalúa el rendimiento de un modelo usando múltiples métricas"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return rmse, mae, r2

def entrenar_y_evaluar_modelo(modelo, nombre, X_train, y_train, X_test, y_test):
    """Entrena un modelo y evalúa su rendimiento"""
    print(f"Entrenando modelo: {nombre}")
    modelo.fit(X_train, y_train)
    predicciones = modelo.predict(X_test)
    rmse, mae, r2 = evaluar_modelo(y_test, predicciones)
    print(f"{nombre} - RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")
    return modelo, (rmse, mae, r2)

def guardar_modelo(modelo, nombre):
    """Guarda un modelo entrenado en disco"""
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{nombre}_{timestamp}.pkl"
    with open(model_output_dir / filename, 'wb') as f:
        pickle.dump(modelo, f)
    print(f"Modelo {nombre} guardado como: {filename}")
    return filename

In [21]:
# 7. Entrenamiento modelos base sin optimización
modelos_base = {
    'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42),
    'LightGBM': LGBMRegressor(n_estimators=100, random_state=42)
}

resultados_base = {}
modelos_guardados = {}

for nombre, modelo in modelos_base.items():
    modelo_entrenado, metricas = entrenar_y_evaluar_modelo(
        modelo, nombre, X_train_scaled, y_train, X_test_scaled, y_test
    )
    resultados_base[nombre] = metricas
    
    # Guardar modelo
    modelo_file = guardar_modelo(modelo_entrenado, nombre)
    modelos_guardados[nombre] = modelo_file

Entrenando modelo: RandomForest


KeyboardInterrupt: 

In [ ]:
# 8. Optimización de hiperparámetros con Optuna (RandomForest)
def objective_rf(trial):
    """Función objetivo para optimización de RandomForest con Optuna"""
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 5, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2']),
        'random_state': 42
    }
    model = RandomForestRegressor(**params)
    score = cross_val_score(model, X_train_scaled, y_train, 
                          scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1)
    return -np.mean(score)  # Devolvemos negativo porque optimizamos minimizando

print("Iniciando optimización de hiperparámetros para RandomForest...")
study_rf = optuna.create_study(direction='minimize')
study_rf.optimize(objective_rf, n_trials=30)

print("\nMejores hiperparámetros encontrados:")
print(study_rf.best_params)

# Entrenar el mejor modelo RandomForest con los parámetros optimizados
best_rf_params = study_rf.best_params
best_rf = RandomForestRegressor(**best_rf_params, random_state=42)
mejor_rf, metricas_rf = entrenar_y_evaluar_modelo(
    best_rf, 'RandomForest_Optuna', X_train_scaled, y_train, X_test_scaled, y_test
)
resultados_base['RandomForest_Optuna'] = metricas_rf

# Guardar el mejor modelo
mejor_rf_file = guardar_modelo(mejor_rf, 'RandomForest_Optuna')
modelos_guardados['RandomForest_Optuna'] = mejor_rf_file

In [ ]:
# 9. Visualización de resultados
resultados_df = pd.DataFrame(resultados_base, index=['RMSE', 'MAE', 'R2']).T

# Visualizar resultados en tabla
display(resultados_df)

# Gráficas de comparación
plt.figure(figsize=(12, 6))
sns.barplot(x=resultados_df.index, y=resultados_df['RMSE'])
plt.title('Comparación de RMSE entre modelos')
plt.ylabel('RMSE (menor es mejor)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(model_output_dir / 'rmse_comparison.png')
plt.show()

# Gráfica de R²
plt.figure(figsize=(12, 6))
sns.barplot(x=resultados_df.index, y=resultados_df['R2'])
plt.title('Comparación de R² entre modelos')
plt.ylabel('R² (mayor es mejor)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(model_output_dir / 'r2_comparison.png')
plt.show()

In [ ]:
# 10. Exportar resultados finales
# Crear un dataset de xarray con resultados
def create_results_dataset(resultados_df):
    """Crea un dataset de xarray con los resultados para exportar como NetCDF"""
    modelo_names = list(resultados_df.index)
    metrics = ['RMSE', 'MAE', 'R2']
    
    # Crear arrays para cada métrica
    rmse_values = resultados_df['RMSE'].values
    mae_values = resultados_df['MAE'].values
    r2_values = resultados_df['R2'].values
    
    # Crear dataset
    ds = xr.Dataset(
        data_vars={
            'RMSE': (['model'], rmse_values),
            'MAE': (['model'], mae_values),
            'R2': (['model'], r2_values)
        },
        coords={
            'model': modelo_names,
        },
        attrs={
            'description': 'Resultados de modelos de predicción de precipitación STHyMOUNTAIN',
            'created': datetime.datetime.now().isoformat(),
            'features_used': ', '.join(feature_cols)
        }
    )
    return ds

# Crear dataset de resultados
results_ds = create_results_dataset(resultados_df)

# Guardar resultados como NetCDF
results_file = model_output_dir / 'model_results.nc'
results_ds.to_netcdf(results_file)
print(f"Resultados guardados como NetCDF en: {results_file}")

# También guardar como CSV para fácil visualización
csv_file = model_output_dir / 'model_results.csv'
resultados_df.to_csv(csv_file)
print(f"Resultados guardados como CSV en: {csv_file}")

# Crear un diccionario con información del modelo para uso futuro
model_info = {
    'date_trained': datetime.datetime.now().isoformat(),
    'feature_columns': feature_cols,
    'target_column': target_column,
    'models_saved': modelos_guardados,
    'results': resultados_df.to_dict(),
    'best_model': resultados_df['RMSE'].idxmin(),
    'scaler': 'scaler.pkl'
}

# Guardar información del modelo
with open(model_output_dir / 'model_info.pkl', 'wb') as f:
    pickle.dump(model_info, f)
print(f"Información del modelo guardada en: {model_output_dir / 'model_info.pkl'}")

print("\n🔥 Entrenamiento completado con éxito! El mejor modelo es:", resultados_df['RMSE'].idxmin())